In [1]:
import pandas as pd
import joblib

# Lade dein bereits trainiertes schnelles Modell
model = joblib.load('random_forest_regression.pkl')

# Apartments-Daten laden
apartments = pd.read_csv('original_apartment_data_analytics_hs24_with_lat_lon.csv')

# Minergie-Daten laden
minergie = pd.read_csv('ogd91_minergiegebaeude_pro_gemeinde.csv')


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# Merge Apartments mit Minergie-Daten
merged_data = apartments.merge(minergie, left_on='bfs_number', right_on='BfsNumber', how='left')

# Gesamt-Minergie-Feature berechnen
merged_data['minergie_total'] = (
    merged_data['Minergie'] +
    merged_data['Minergie_Eco'] +
    merged_data['Minergie_A'] +
    merged_data['Minergie_A_Eco'] +
    merged_data['Minergie_P'] +
    merged_data['Minergie_P_Eco']
)

# Fülle fehlende Werte, falls nötig
merged_data['minergie_total'] = merged_data['minergie_total'].fillna(merged_data['minergie_total'].median())


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

features = merged_data[['area', 'rooms', 'lat', 'lon', 'minergie_total']]
target = merged_data['price']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Schnelles Training
model = RandomForestRegressor(n_estimators=10, max_depth=8, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Modell speichern
joblib.dump(model, 'random_forest_regression_with_minergie.pkl')


['random_forest_regression_with_minergie.pkl']

In [12]:
import joblib
joblib.dump(model, 'model_minergie.pkl')


['model_minergie.pkl']

In [14]:
import gradio as gr
import pandas as pd
import joblib

# Neues Modell mit Minergie laden
model = joblib.load('random_forest_regression_with_minergie.pkl')

# Datensätze laden
apartments = pd.read_csv("original_apartment_data_analytics_hs24_with_lat_lon.csv")
minergie = pd.read_csv("ogd91_minergiegebaeude_pro_gemeinde.csv")
minergie['minergie_total'] = (
    minergie['Minergie'] + minergie['Minergie_Eco'] +
    minergie['Minergie_A'] + minergie['Minergie_A_Eco'] +
    minergie['Minergie_P'] + minergie['Minergie_P_Eco']
)

# Gemeinden vorbereiten (für Dropdown-Menü)
gemeinden_df = apartments[['bfs_number', 'town']].drop_duplicates().sort_values('town')
gemeinde_options = dict(zip(gemeinden_df['town'], gemeinden_df['bfs_number']))

# Vorhersagefunktion definieren
def predict_price(area, rooms, gemeinde_name):
    bfs_number = gemeinde_options[gemeinde_name]

    # Minergie-Wert abrufen
    minergie_total = minergie.loc[minergie['BfsNumber'] == bfs_number, 'minergie_total'].values
    minergie_total = minergie_total[0] if len(minergie_total) else minergie['minergie_total'].median()

    # Durchschnittliche Koordinaten der Gemeinde ermitteln
    coords = apartments.loc[apartments['bfs_number'] == bfs_number, ['lat', 'lon']].mean()

    input_df = pd.DataFrame([{
        'area': area,
        'rooms': rooms,
        'lat': coords['lat'],
        'lon': coords['lon'],
        'minergie_total': minergie_total
    }])

    # Vorhersage mit Modell
    pred = model.predict(input_df)[0]

    return f"💡 Geschätzter Preis: CHF {pred:.2f}"

# Interface erstellen
iface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label="Fläche (m²)"),
        gr.Number(label="Anzahl Zimmer"),
        gr.Dropdown(choices=list(gemeinde_options.keys()), label="Gemeinde")
    ],
    outputs="text",
    title="Immobilienpreisvorhersage inkl. Minergie",
    description="Gib Fläche, Zimmeranzahl und Gemeinde an, um den Wohnungspreis vorherzusagen."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
